In [9]:
!pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.3/202.3 kB 3.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [10]:
#amedasから最新の気象データを取得する

#import dependencies
import requests
import pandas as pd
from datetime import datetime,timedelta

#data1: 地点情報
url = 'https://www.jma.go.jp/bosai/amedas/const/amedastable.json'
r = requests.get(url)
data1 = r.json()

#data2: amedasの最新気象データ
##最新の時刻
r = requests.get('https://www.jma.go.jp/bosai/amedas/data/latest_time.txt')
latest_time = r.text
latest_time = datetime.strptime(latest_time, '%Y-%m-%dT%H:%M:%S%z').strftime('%Y%m%d%H0000')
print(latest_time)

##最新のデータ
url = f"https://www.jma.go.jp/bosai/amedas/data/map/{latest_time}.json"
r = requests.get(url)
data2 = r.json()

#data1とdata2を統合
amedas_data = {key: {**data1.get(key, {}), **data2.get(key, {})} for key in set(data1) | set(data2)}

#データ整形
amedas = pd.DataFrame(amedas_data)
amedas = amedas.T.rename_axis('amdno').reset_index()

##geo-cordをととのえる
amedas[['lat','lon']] = amedas[['lat','lon']].applymap(lambda x: x[0] + x[1]/60)

##残りのリスト形式のデータから値のみ抽出
amedas = amedas.applymap(lambda x: x[0] if type(x)==list else x)

#amedas.to_csv('./data/amedas_latest_data.csv',encoding='utf-8-sig',index=False)

20230728170000


In [13]:
amedas.shape

(1287, 28)

In [14]:
amedas.dropna(subeset=['temp'])

TypeError: DataFrame.dropna() got an unexpected keyword argument 'subeset'